<a href="https://colab.research.google.com/github/hmsjaber/bootstrap_project/blob/master/Torrent_to_google_drive_Jaber.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python -m pip install --upgrade pip setuptools wheel
!python -m pip install lbry-libtorrent


In [2]:
from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import libtorrent as lt
import time
import datetime

ses = lt.session()
ses.listen_on(6881, 6891)
downloads = []
from google.colab import files

source = files.upload()
params = {
    "save_path": "/content/drive/folder location",
    "ti": lt.torrent_info(list(source.keys())[0]),
}
downloads.append(ses.add_torrent(params))

In [ ]:
import libtorrent as lt
import time
import datetime

ses = lt.session()
ses.listen_on(6881, 6891)
downloads = []
params = {"save_path": "/content/drive/folder location"}

while True:
    magnet_link = input("Enter Magnet Link Or Type Exit: ")
    if magnet_link.lower() == "exit":
        break
    downloads.append(
        lt.add_magnet_uri(ses, magnet_link, params)
    )

In [ ]:
from IPython.display import display
import ipywidgets as widgets

state_str = [
    "queued",
    "checking",
    "downloading metadata",
    "downloading",
    "finished",
    "seeding",
    "allocating",
    "checking fastresume",
]

layout = widgets.Layout(width="auto")
style = {"description_width": "initial"}
download_bars = [
    widgets.FloatSlider(
        step=0.01, disabled=True, layout=layout, style=style
    )
    for _ in downloads
]
display(*download_bars)

while downloads:
    next_shift = 0
    for index, download in enumerate(downloads[:]):
        bar = download_bars[index + next_shift]
        if not download.is_seed():
            s = download.status()

            bar.description = " ".join(
                [
                    download.name(),
                    str(s.download_rate / 1000),
                    "kB/s",
                    state_str[s.state],
                ]
            )
            bar.value = s.progress * 100
        else:
            next_shift -= 1
            ses.remove_torrent(download)
            downloads.remove(download)
            bar.close() # Seems to be not working in Colab (see https://github.com/googlecolab/colabtools/issues/726#issue-486731758)
            download_bars.remove(bar)
            print(download.name(), "complete")
    time.sleep(1)

